In [ ]:
import mediapipe as mp
import tensorflow as tf
print(tf.__version__)  # 2.10.1
print(mp.solutions.hands)  # 정상 작동 여부 확인


2.10.0
<module 'mediapipe.python.solutions.hands' from 'c:\\Users\\chaeyeonhan\\anaconda3\\envs\\slr_test\\lib\\site-packages\\mediapipe\\python\\solutions\\hands.py'>


In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\chaeyeonhan\OneDrive\2025-1\capstone_design\models\Korean\LSTM+GRU_pca.keras")

print("입력 shape:", model.input_shape)
print("출력 shape:", model.output_shape)


입력 shape: (None, 60, 450)
출력 shape: (None, 100)


In [ ]:
DATASET_PATH = "C:\\Users\\chaeyeonhan\\OneDrive\\2025-1\\capstone_design"
VIDEO_PATH = f"{DATASET_PATH}/data/top100_videos_only"
LABEL_EXCEL = f"{DATASET_PATH}/data/KETI_top100_clean_words.xlsx"

In [ ]:
import pandas as pd
import os
import unicodedata
import numpy as np
def load_label_info(label_excel_path):
    label_df = pd.read_excel(label_excel_path)
    label_df['파일ID'] = label_df['파일명'].apply(lambda x: os.path.splitext(x)[0])
    video_to_label = dict(zip(label_df['파일ID'], label_df['한국어']))
    labels = sorted(unicodedata.normalize("NFC", l) for l in label_df['한국어'].unique())
    label_map = {label: idx for idx, label in enumerate(labels)}
    return label_df, label_map, video_to_label

label_df, label_map, video_to_label = load_label_info(LABEL_EXCEL)

In [ ]:
# 손 인식 여부 판단 함수
def is_hand_visible(landmarks):
    return landmarks is not None

In [ ]:
#한국어 로드
from PIL import ImageFont, ImageDraw, Image

font_path =r"C:\Users\chaeyeonhan\OneDrive\2025-1\capstone_design\NotoSansKR-Medium.ttf"
font = ImageFont.truetype(font_path, 32)

def draw_text_hangul(img, text, org=(20, 20), font=font):

    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    draw.text(org, text, font=font, fill=(255, 255, 255))  # 하얀색
    return np.array(img_pil)


In [ ]:
def normalize_single_frame(frame):
    return (frame - np.mean(frame)) / (np.std(frame) + 1e-6)


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import joblib
from collections import deque
import time

#모델 & PCA 로드-KETI
model = tf.keras.models.load_model(r"C:\Users\chaeyeonhan\OneDrive\2025-1\capstone_design\models\Korean\LSTM+GRU.keras")
pca = joblib.load(r"C:\Users\chaeyeonhan\OneDrive\2025-1\capstone_design\test\KETI\pca_450.joblib")

# 설정
SEQ_LENGTH = 60
NUM_FEATURES = 450
index_to_label = {v: k for k, v in label_map.items()}
classes = [index_to_label[i] for i in range(len(index_to_label))]



# MediaPipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)


def extract_keypoints(results):
    pose = []
    face = []
    lh = []
    rh = []

    if results.pose_landmarks:
        for res in results.pose_landmarks.landmark:
            pose.extend([res.x, res.y, res.z])
    else:
        pose = [0] * 33 * 3

    if results.face_landmarks:
        for res in results.face_landmarks.landmark:
            face.extend([res.x, res.y, res.z])
    else:
        face = [0] * 468 * 3

    if results.left_hand_landmarks:
        for res in results.left_hand_landmarks.landmark:
            lh.extend([res.x, res.y, res.z])
    else:
        lh = [0] * 21 * 3

    if results.right_hand_landmarks:
        for res in results.right_hand_landmarks.landmark:
            rh.extend([res.x, res.y, res.z])
    else:
        rh = [0] * 21 * 3

    return np.array(pose + face + lh + rh)

# 6. 웹캠 시작
cap = cv2.VideoCapture(0)
sequence = deque(maxlen=SEQ_LENGTH)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Mediapipe 처리
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = holistic.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 랜드마크 시각화
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # 특징 추출 및 시퀀스 저장
    keypoints = extract_keypoints(results)
    keypoints = normalize_single_frame(keypoints)
    #keypoints_pca = pca.transform([keypoints])[0]
    sequence.append(keypoints)

    # 60프레임 누적 시 예측
    if len(sequence) == 60:
        print(" Input shape:", np.array(sequence).shape)
        print(" Input sample (first 5 dims):", sequence[-1][:5])
        input_seq = np.expand_dims(sequence, axis=0)
        y_pred = model.predict(input_seq, verbose=0)
        class_idx = np.argmax(y_pred)
        confidence = y_pred[0][class_idx]
        label = classes[class_idx]
        print(" Predicted probs:", y_pred[0])

        if confidence > 0.3:
            frame = draw_text_hangul(frame, f"{label} ({confidence:.2f})", org=(50, 50))

    # 화면 출력
    cv2.imshow("Sign Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC키 종료
        break

cap.release()
cv2.destroyAllWindows()

🔍 Input shape: (60, 450)
🔍 Input sample (first 5 dims): [ -2.64892303 -11.07984905  -1.41570189  13.27951242   5.19043725]
🔍 Predicted probs: [0.01377518 0.0733958  0.00326515 0.01019099 0.01166015 0.00104897
 0.00311695 0.0003883  0.01607395 0.00837193 0.01143987 0.00419948
 0.02115472 0.12441118 0.00104453 0.00538477 0.00131938 0.01328824
 0.00131703 0.10210387 0.00199162 0.01311427 0.00721661 0.00506952
 0.00035402 0.00203619 0.02178597 0.0029688  0.00043927 0.00361667
 0.00122774 0.00047515 0.0016646  0.00279066 0.02053896 0.00520878
 0.01138615 0.00216457 0.00586697 0.00146491 0.00714324 0.00310472
 0.00303355 0.00102835 0.00202589 0.00296414 0.0038977  0.01265416
 0.00100096 0.0099616  0.00385276 0.00108028 0.00037208 0.00797107
 0.00355775 0.01244811 0.0048943  0.00489145 0.00222444 0.00611637
 0.00186476 0.00317407 0.00372413 0.00082277 0.00361734 0.00464906
 0.00072597 0.00956598 0.00502336 0.00183008 0.00179271 0.0007829
 0.00247463 0.01561425 0.00261074 0.00241033 0.00542356

KeyboardInterrupt: 

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라 열기 실패")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패")
        break
    cv2.imshow("Webcam Test", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch

print("🔹 PyTorch GPU 연동 테스트")
print("CUDA 사용 가능:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))
    print("CUDA 버전:", torch.version.cuda)
else:
    print(" GPU를 사용하지 못하고 있어요.")


OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다. Error loading "c:\Users\chaeyeonhan\anaconda3\envs\slr_test\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
import importlib
import platform

def check_package(name):
    try:
        module = importlib.import_module(name)
        version = getattr(module, '__version__', '버전 정보 없음')
        print(f" {name} 설치됨 (버전: {version})")
    except ImportError:
        print(f" {name} 설치 안 됨")

print(" Python 버전:", platform.python_version())
print()

# 필수 패키지 확인
packages = ['protobuf', 'torch', 'tensorflow', 'mediapipe']
for pkg in packages:
    check_package(pkg)

# CUDA 사용 가능 여부 확인 (torch 기준)
try:
    import torch
    print("  CUDA 사용 가능 여부:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("  GPU 이름:", torch.cuda.get_device_name(0))
except ImportError:
    pass


🧠 Python 버전: 3.8.20

❌ protobuf 설치 안 됨
✅ torch 설치됨 (버전: 1.12.1+cu113)
✅ tensorflow 설치됨 (버전: 2.10.0)
✅ mediapipe 설치됨 (버전: 0.10.5)
⚙️  CUDA 사용 가능 여부: True
🖥️  GPU 이름: NVIDIA GeForce RTX 4060


In [ ]:
import platform
import torch
import tensorflow as tf
import mediapipe as mp
import google.protobuf

print("🐍 Python:", platform.python_version())

print("\n🔹 TensorFlow:", tf.__version__)
print("🔹 TensorFlow GPU 사용 가능:", tf.config.list_physical_devices('GPU'))

print("\n🔥 Torch:", torch.__version__)
print("🔥 CUDA 사용 가능:", torch.cuda.is_available())

print("\n🧠 Mediapipe:", mp.__version__)
print("📦 Protobuf:", google.protobuf.__version__)


c:\Users\chaeyeonhan\anaconda3\envs\slr_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🐍 Python: 3.8.20

🔹 TensorFlow: 2.10.0
🔹 TensorFlow GPU 사용 가능: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

🔥 Torch: 1.12.1+cu113
🔥 CUDA 사용 가능: True

🧠 Mediapipe: 0.10.5
📦 Protobuf: 3.20.3


In [ ]:
import tensorflow as tf

print("\n🔹 TensorFlow GPU 연동 테스트")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("사용 가능한 GPU 장치:", gpus)
    print("TensorFlow GPU 이름:", tf.test.gpu_device_name())
else:
    print("TensorFlow에서 GPU를 인식하지 못해요.")



🔹 TensorFlow GPU 연동 테스트
사용 가능한 GPU 장치: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow GPU 이름: /device:GPU:0
